In [ ]:
%matplotlib inline
import pandas as pd
import operator
import psycopg2
import pylab
import numpy as np
import datetime
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.patches as mpatches
from scipy.stats.stats import pearsonr
import matplotlib.lines as mlines
import matplotlib as mpl
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from statsmodels.tsa import stattools
import statsmodels.api as sm
import scipy
import random
import seaborn as sns
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
import re
from collections import OrderedDict
import statsmodels.api as sm
from scipy import stats
import statsmodels
from statsmodels.graphics.api import qqplot

mpl.rcdefaults()
pd.options.display.mpl_style = 'default'

In [ ]:
# Read database parameters from default_profile
dbitems = {}
with open('default_profile') as f:
    for line in f.readlines():
        item = line.split(" ")[1].split("=")
        dbitems[item[0]] = item[1].strip()
        
# Connect to database with psycopg2
try:
    conn = psycopg2.connect("dbname='%s' user='%s' host='%s' password='%s'"%(dbitems['PGDATABASE'],dbitems['PGUSER'],dbitems['PGHOST'],dbitems['PGPASSWORD']))
except:
    print "Unable to connect to the database"

# Hour of Day Heatmap

In [ ]:
sem_df = pd.read_sql_query("SELECT incident, time_month, time_day, time_hour FROM semantic.master WHERE time_year = 2015", conn)

In [ ]:
sem_df['day_of_week'] = sem_df.apply(lambda x: datetime.datetime(2015, int(x.time_month), int(x.time_day)).weekday(), axis=1)

In [ ]:
df_hour = sem_df.groupby(['day_of_week', 'time_hour']).count()

In [ ]:
df_hour

In [ ]:
hm_hour = np.zeros((7,24))

In [ ]:
for i in range(7):
    for j in range(24):
        hm_hour[i][j] = df_hour.ix[(i,(j+7)%24)].iloc[0]/52.0

In [ ]:
disc = raw_input('Discrete? ').lower()
if disc != 'yes':
    grid = raw_input('Include Grid? ').lower()
else:
    grid = 'yes'

mpl.rcdefaults()
plt.figure(figsize=(20,10))

if disc == 'yes':
    plt.imshow(hm_hour, interpolation='none')
else:
    plt.imshow(hm_hour)

plt.yticks(range(7),['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], fontsize = 20)
plt.xticks(range(24),range(7,24) + range(0,7), fontsize = 20)
cb = plt.colorbar(orientation = 'horizontal')
cb.ax.tick_params(labelsize=15) 

if  grid == 'yes':
    for i in range(7):
        plt.axhline(i-0.5, color = 'k', linewidth = 2)
    for i in range(24):
        plt.axvline(i-0.5, color = 'k', linewidth = 2)


# Day of Month Heatmap

In [ ]:
df_day = sem_df.groupby(['time_month', 'time_day']).count()

In [ ]:
df_day

In [ ]:
hm_day = np.zeros((12,31))

In [ ]:
for i in range(1,13):
    for j in range(1,32):
        try:
            hm_day[i-1][j-1] = df_day.ix[(i,j)].iloc[0]
            if hm_day[i-1][j-1] > 300:
                hm_day[i-1][j-1] = np.nan
        except IndexError:
            hm_day[i-1][j-1] = np.nan

In [ ]:
disc = raw_input('Discrete? ').lower()
if disc != 'yes':
    grid = raw_input('Include Grid? ').lower()
else:
    grid = 'yes'

mpl.rcdefaults()
plt.figure(figsize=(20,10))
if disc == 'yes':
    plt.imshow(hm_day, interpolation='none')
else:
    plt.imshow(hm_day)
plt.yticks(range(12),['Jan', 'Feb', 'Mar', 'Apr', 'May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], fontsize = 20)
plt.xticks(range(31),range(1,32), fontsize = 20)
cb = plt.colorbar(orientation = 'horizontal')
cb.ax.tick_params(labelsize=15) 

if  grid == 'yes':
    for i in range(12):
        plt.axhline(i-0.5, color = 'k', linewidth = 2)
    for i in range(31):
        plt.axvline(i-0.5, color = 'k', linewidth = 2)
